In [33]:
from dotenv import load_dotenv
import os
from langchain_ollama import ChatOllama


# Load environment variables from the .env file
load_dotenv()


True

In [36]:
# initialize llm
def initialize_llm(model_name="llama3.2:3b", temperature=0, max_tokens=50):
    """
    Initializes the ChatOllama model with specified parameters.
    Returns:
        ChatOllama: Instance of ChatOllama model.
    """
    try:
        llm = ChatOllama(
            model=model_name,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        print("Model initialized successfully.")
        return llm
    except Exception as e:
        print(f"Error initializing the model: {e}")
        return None
# Initialize the model
model = initialize_llm()


Model initialized successfully.


* ask something related to external system

In [38]:
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

response = model.invoke([HumanMessage(content="hey what's the time")]) 
print(parser.invoke(response))

I'm not sure what time it is, as I'm a text-based AI and don't have real-time access to your location. However, I can suggest some ways for you to find out the current time:

1. Check your phone or computer's clock.
2. Look at a physical clock or watch.
3. Search for "current time" on a search engine like Google.

If you need help with anything else, feel free to ask!


### implement llm function calling/too calling


In [39]:
from langchain_core.tools import tool


In [51]:
from datetime import datetime, date

# create fucnction/tool 

# get current time
@tool
def get_current_time() -> str:
    """
    Returns the current time as a formatted string like 'HH:MM:SS AM/PM', in 12-hour format with AM/PM notation.
    
    Returns:
        str: A string representing the current time in the format 'HH:MM:SS AM/PM'.
    """
    try:
        # Get the current date and time
        now = datetime.now()

        # Format as a string in 12-hour format with AM/PM notation
        time_str = now.strftime("%I:%M:%S %p")

        # Return the formatted string
        return time_str
    except Exception as e:
        # Return an error message if an exception occurs
        return f"An error occurred while fetching the time: {e}"
@tool 
def get_weather(location: str) -> dict:
    """
    Fetches the current weather data for a specified city

    Args:
        location (str): Location for weather query (e.g., city name).

    Returns:
        dict: A dummy weather forecast response.
    """
    # Dummy response
    return {
        "location": location,
        "temperature": "25°C",
        "condition": "Sunny",
        "humidity": "60%",
        "wind_speed": "15 km/h"
    }    

# get current date
@tool
def current_date() -> str:
    """
    Return the current date in YYYY-MM-DD format.
    
    Returns:
        str: A string representing the current date in the format 'YYYY-MM-DD'.
    """
    try:
        return date.today().strftime("%Y-%m-%d")
    except Exception as e:
        return f"Error occurred while fetching the date: {e}"



In [52]:
tools = [
         get_current_time,
         current_date,
         get_weather
        ]

In [53]:
# Bind the tools to the LLM
llm_with_tools = model.bind_tools(tools)

In [54]:
response = llm_with_tools.invoke([HumanMessage(content="hey what's the time")]) 


In [55]:
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (fb112caa-a2b9-45bc-a6b3-06e4cfe3495d)
 Call ID: fb112caa-a2b9-45bc-a6b3-06e4cfe3495d
  Args:


In [56]:
response = llm_with_tools.invoke([HumanMessage(content="hey what's weather in delhi")]) 
response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_weather (13cebafd-5e37-472e-9dfa-87fec637a4ae)
 Call ID: 13cebafd-5e37-472e-9dfa-87fec637a4ae
  Args:
    location: delhi
